# <div align="center">Trabalho Computacional AV2: Modelos de Regressão e Classificação</div>

Equipe:

<ul>
    <li>Guilherme de Farias</li>
    <li>Tiago Farias</li>
</ul>

Bibliotecas utilizadas:

In [1]:
import pandas as pd
import numpy as np
import plotly.express as exp
import plotly.graph_objects as go

pd.set_option('display.float_format', '{:.2f}'.format)

## <div align="center">Introdução</div>

O presente trabalho é composto por duas etapas em que deve-se utilizar os conceitos de IA baseados em modelos preditivos que realizam seu processo de aprendizagem através da minimização de uma função **custo (loss function)**. Em ambas etapas do trabalho, tais modelos utilizam o paradigma supervisionado para aprender a partir dos pares, vetor de características (variáveis regressoras) e variável dependente. Contudo, a tarefa da primeira etapa trata-se do desenvolvimento de um sistema que faz predições quantitativas (**regressão**), ao passo que a segunda etapa é caracterizada pelo desenvolvimento de um sistema que realiza predições qualitativas (**classificação**).

## <div align="center">Tarefa de Regressão. [3,0 pts]</div>

Para o problema de **regressão** solicita-se que faça o acesso ao conjunto de dados disponibilizado na plataforma AVA, chamado **aerogerador.dat**. A variável independente é uma medida de velocidade do vento, e a variável dependente é uma observação de potência gerada pelo aerogerador.

### Etapa 1

Faça uma visualização inicial dos dados através do **gráfico de espalhamento**. Nessa etapa, faça discussões sobre quais serão as características de um modelo que consegue entender o padrão entre variáveis regressoras e variáveis observadas.

In [2]:
# lendo os dados do arquivo `dados/aerogerador.dat`
dados: pd.DataFrame = (
    pd.read_csv(
        "dados/aerogerador.dat", 
        sep=r'\s+|\t', # corresponder a uma ou mais ocorrências de espaços em branco (\s+) ou uma única tabulação (\t)
        engine='python', # a engine c, que é padrão, não suporta lidar com regex.
        header=None, 
        names=("velocidade do vento", "potência gerada")
    )
)

In [3]:
fig = exp.scatter(
    dados,
    x='velocidade do vento',
    y='potência gerada',
    title="Velocidade do Vento x Potência Gerada",
    labels={
        'velocidade do vento': 
            'Velocidade do Vento (m/s)', 
        'potência gerada': 
            'Potência Gerada (W)'
    }
)

fig.show()

### Etapa 2

Em seguida, organize os dados de modo que as variáveis regressoras sejam armazenadas em uma matriz **(X)** de dimensão **ℝⁿˣᵖ**. Faça o mesmo para o vetor de variável dependente **(y)**, organizando em um vetor de dimensão **ℝⁿˣ¹**.

In [4]:
x: np.ndarray = np.array(dados['velocidade do vento']) # Variável INdependente
y: np.ndarray = np.array(dados['potência gerada']) # Variável dependente

# Reformatação para garantir que x e y sejam matrizes colunas
x = x.reshape(-1, 1) 
y = y.reshape(-1, 1) 
# -1 é a mesma coisa que len(x) ou len(y) nesses casos

# Adicionando uma coluna de 1s para o cálculo do intercepto (termo constante)
X: np.ndarray = np.concatenate(
    [np.ones((len(x), 1)), x], 
    axis=1
)

### Etapas 3 e 4

Os modelos a serem implementados nessa etapa serão: **MQO tradicional**, **MQO regularizado** (Tikhonov) e **Média de valores observáveis**. **Obs**: lembre-se que todos os modelos também estimam o valor do intercepto.

Para o modelo regularizado, há a dependência da definição de seu hiperparâmetro λ. Assim, sua equipe deve testar o presente modelo para os seguintes valores de lambda:

λ = {0, 0.25, 0.5, 0.75, 1}

Assim, ao todo, existirão 6 estimativas diferentes do vetor β ∈ ℝ^{p+1×1}

#### MQO tradicional

In [5]:
# Estimação dos coeficientes usando a fórmula do MMQO tradicional (beta = (X^T * X)^-1 * X^T * y)
B: np.ndarray = np.linalg.pinv(X.T @ X) @ X.T @ y

# Coeficientes da regressão
intercepto: float = B[0][0]
coeficiente_angular: float = B[1][0]

# Gerando pontos para a linha de regressão
x_pred: np.ndarray = np.linspace(min(x), max(x), 100)
X_pred: np.ndarray = np.concatenate([np.ones((len(x_pred), 1)), x_pred.reshape(-1, 1)], axis=1)
y_pred: np.ndarray = X_pred @ B

# Exibindo os coeficientes estimados
print(f"Intercepto (beta_0): {intercepto:.4f}")
print(f"Coeficiente angular (beta_1): {coeficiente_angular:.4f}")

Intercepto (beta_0): -217.6903
Coeficiente angular (beta_1): 56.4439


In [6]:
# Adicionando a linha de regressão ao gráfico
fig.add_trace(
    go.Scatter(
        x=x_pred.flatten(),
        y=y_pred.flatten(),
        mode='lines',
        name='Linha de Regressão MQO tradicional',
        line={'color':'tan'}
    )
)

fig.show()

# removendo o traço da figura
fig.data = fig.data[:-1]

#### MQO regularizado (Tikhonov)

In [7]:
I: np.ndarray = np.eye(X.shape[1])

hiperparametros: tuple[float] = 0, .25, .5, .75, 1
cores: tuple[str] = "tan", "darkblue", "teal", "firebrick", "darkviolet"

for hp, cor in zip(hiperparametros, cores):
    print(f'{'-'*8}HIPERPARÂMETRO = {hp}{'-'*8}')
    # Estimação dos coeficientes usando a fórmula do MQO regularizado (beta = ((X^T * X) + λI)^-1 * X^T * y)
    B: np.ndarray = np.linalg.pinv((X.T @ X) + hp * I) @ X.T @ y

    # Coeficientes da regressão
    intercepto: float = B[0][0]
    coeficiente_angular: float = B[1][0]

    # Gerando pontos para a linha de regressão
    x_pred: np.ndarray = np.linspace(min(x), max(x), 100)
    X_pred: np.ndarray = np.concatenate([np.ones((len(x_pred), 1)), x_pred.reshape(-1, 1)], axis=1)
    y_pred: np.ndarray = X_pred @ B

    # Exibindo os coeficientes estimados
    print(f"Intercepto (beta_0): {intercepto:.4f}")
    print(f"Coeficiente angular (beta_1): {coeficiente_angular:.4f}")
    
    fig.add_trace(
        go.Scatter(
            x=x_pred.flatten(),
            y=y_pred.flatten(),
            mode='lines',
            name=f'Linha de Regressão (MQO Regularizado com hiperparâmetro = {hp})',
            line={'color': f'{cor}'}
        )
    )
    
    print(''*16)
    
fig.show()

fig.data = fig.data[:1]

--------HIPERPARÂMETRO = 0--------
Intercepto (beta_0): -217.6903
Coeficiente angular (beta_1): 56.4439

--------HIPERPARÂMETRO = 0.25--------
Intercepto (beta_0): -217.0270
Coeficiente angular (beta_1): 56.3739

--------HIPERPARÂMETRO = 0.5--------
Intercepto (beta_0): -216.3676
Coeficiente angular (beta_1): 56.3044

--------HIPERPARÂMETRO = 0.75--------
Intercepto (beta_0): -215.7122
Coeficiente angular (beta_1): 56.2354

--------HIPERPARÂMETRO = 1--------
Intercepto (beta_0): -215.0607
Coeficiente angular (beta_1): 56.1667



#### Média de valores observáveis

In [8]:
# Cálculo das médias de x e y
x_media = np.mean(x)
y_media = np.mean(y)

# Estimação dos coeficientes usando a média dos valores observáveis
# coeficiente angular (beta_1) é calculado como a razão entre a covariância e a variância de x
coeficiente_angular = np.sum((x - x_media) * (y - y_media)) / np.sum((x - x_media) ** 2)

# Intercepto (beta_0) calculado para que a linha passe pela média dos valores
intercepto = y_media - coeficiente_angular * x_media

# Gerando pontos para a linha de regressão
x_pred = np.linspace(min(x), max(x), 100)
y_pred = intercepto + coeficiente_angular * x_pred

# Exibindo os coeficientes estimados
print(f"Intercepto (beta_0): {intercepto:.4f}")
print(f"Coeficiente angular (beta_1): {coeficiente_angular:.4f}")

Intercepto (beta_0): -217.6903
Coeficiente angular (beta_1): 56.4439


In [9]:
# Adicionando a linha de regressão ao gráfico
fig.add_trace(
    go.Scatter(
        x=x_pred.flatten(),
        y=y_pred.flatten(),
        mode='lines',
        name='Linha de Regressão (Média de Valores Observáveis)',
        line={'color': 'tan'}
    )
)

fig.show()

# removendo o traço da figura
fig.data = fig.data[:-1]

### Etapas 5 e 6

Para validar os modelos utilizados na tarefa de regressão, sua equipe deve projetar a validação utilizando as simulações por Monte Carlo. Nessa etapa, defina a quantidade de rodadas da simulação igual a \( R = 500 \). Em cada rodada, deve-se realizar o particionamento em 80% dos dados para treinamento e 20% para teste. A medida de desempenho de cada um dos 5 modelos diferentes deve ser a soma dos desvios quadráticos (RSS - *Residual Sum of Squares*) e cada medida obtida deve ser armazenada em uma lista.

Ao final das \( R \) rodadas calcule para cada modelo utilizado, média aritmética, desvio-padrão, valor maior, valor menor de cada RSS. Coloque os resultados obtidos em uma tabela e **discuta os resultados obtidos**. 

**Obs:** O resultado não precisa ser limitado a tabela, como pode ser expresso via gráficos!


In [12]:
# Definindo o número de rodadas da simulação
R: int = 500

# Inicializar listas para armazenar o RSS de cada modelo em cada rodada
rss_mqoTradicional: list[float] = []
rss_mqoRegularizado_0: list[float] = []
rss_mqoRegularizado_025: list[float] = []
rss_mqoRegularizado_05: list[float] = []
rss_mqoRegularizado_075: list[float] = []
rss_mqoRegularizado_1: list[float] = []
rss_media: list[float] = []

# número da última amostra de treinamento
n80: int = int(len(dados) * 0.8)

# Loop de simulação de Monte Carlo
for _ in range(R):
    # Amostra embaralhada de dados para cada iteração
    dados_embaralhados: pd.DataFrame = dados.sample(frac=1, ignore_index=True)
    
    dados_treinamento: pd.DataFrame = dados_embaralhados.iloc[:n80]
    dados_validacao: pd.DataFrame = dados_embaralhados.iloc[n80:]
    
    # Separação das variáveis de treinamento e validação
    x_treinamento: np.ndarray = np.array(dados_treinamento['velocidade do vento']).reshape(-1, 1)
    y_treinamento: np.ndarray = np.array(dados_treinamento['potência gerada']).reshape(-1, 1)
    x_validacao: np.ndarray = np.array(dados_validacao['velocidade do vento']).reshape(-1, 1)
    y_validacao: np.ndarray = np.array(dados_validacao['potência gerada']).reshape(-1, 1)

    # Adicionando coluna de 1s para cálculo do intercepto
    X_treinamento: np.ndarray = np.concatenate([np.ones((len(x_treinamento), 1)), x_treinamento], axis=1)
    X_validacao: np.ndarray = np.concatenate([np.ones((len(x_validacao), 1)), x_validacao], axis=1)
    
    # Estimação dos coeficientes para cada modelo e cálculo do RSS no conjunto de teste
    
    # MQO tradicional
    B_trad: np.ndarray = (
        np.linalg.pinv(X_treinamento.T @ X_treinamento) @ X_treinamento.T @ y_treinamento
    )
    y_pred_trad: np.ndarray = X_validacao @ B_trad
    rss_mqoTradicional.append(float(np.sum((y_validacao - y_pred_trad) ** 2)))
    
    # MQO regularizado com diferentes hiperparâmetros
    for hp, lista_rss in zip(hiperparametros, [rss_mqoRegularizado_0, rss_mqoRegularizado_025, rss_mqoRegularizado_05, rss_mqoRegularizado_075, rss_mqoRegularizado_1]):
        B_reg: np.ndarray = (
            np.linalg.pinv((X_treinamento.T @ X_treinamento) + hp * I) @ X_treinamento.T @ y_treinamento
        )
        y_pred_reg: np.ndarray = X_validacao @ B_reg
        lista_rss.append(float(np.sum((y_validacao - y_pred_reg) ** 2)))

    # Média dos valores observáveis
    x_treinamento_media: float = float(np.mean(x_treinamento))
    y_treinamento_media: float = float(np.mean(y_treinamento))
    coef_angular: float = float(
        np.sum((x_treinamento - x_treinamento_media) * (y_treinamento - y_treinamento_media)) / np.sum((x_treinamento - x_treinamento_media) ** 2)
    )
    intercepto_media: float = float(y_treinamento_media - coef_angular * x_treinamento_media)
    y_pred_media: np.ndarray = intercepto_media + coef_angular * x_validacao
    rss_media.append(float(np.sum((y_validacao - y_pred_media) ** 2)))

# Resultados finais: listas de RSS para cada modelo em cada rodada
rss_results: dict[str, list[float]] = {
    "RSS Tradicional": rss_mqoTradicional,
    "RSS Regularizado λ=0": rss_mqoRegularizado_0,
    "RSS Regularizado λ=0.25": rss_mqoRegularizado_025,
    "RSS Regularizado λ=0.5": rss_mqoRegularizado_05,
    "RSS Regularizado λ=0.75": rss_mqoRegularizado_075,
    "RSS Regularizado λ=1": rss_mqoRegularizado_1,
    "RSS Média de Valores Observáveis": rss_media,
}

# Convertendo para DataFrame para facilitar a visualização
df_rss_results: pd.DataFrame = pd.DataFrame(rss_results)
df_rss_results.describe()

,RSS Tradicional,RSS Regularizado λ=0,RSS Regularizado λ=0.25,RSS Regularizado λ=0.5,RSS Regularizado λ=0.75,RSS Regularizado λ=1,RSS Média de Valores Observáveis
count,500.00,500.00,500.00,500.00,500.00,500.00,500.00
mean,359338.19,359338.19,359305.90,359297.13,359311.36,359348.07,359338.19
std,77760.34,77760.34,77243.50,76735.70,76236.79,75746.60,77760.34
min,193477.54,193477.54,193876.82,194291.99,194722.69,195168.54,193477.54
25%,303664.87,303664.87,304135.05,304463.86,305178.02,305628.34,303664.87
50%,352746.73,352746.73,352619.13,352882.32,352928.93,352595.12,352746.73
75%,406336.06,406336.06,406304.55,406176.68,406195.23,405775.24,406336.06
max,652798.39,652798.39,650827.22,648903.32,647025.80,645193.75,652798.39


In [13]:
# Criando um gráfico de barras para efeito de visualização

# Usando os dados do describe() para visualização
df_stats = df_rss_results.describe().T  # Transpor para que as estatísticas sejam colunas

# Criar um gráfico de barras com as estatísticas 'mean', 'std', 'min' e 'max' para cada modelo
fig = go.Figure()

# Adicionando as estatísticas ao gráfico
fig.add_trace(go.Bar(
    x=df_stats.index, 
    y=df_stats['mean'], 
    name='Média'
))

fig.add_trace(go.Bar(
    x=df_stats.index, 
    y=df_stats['std'], 
    name='Desvio Padrão'
))

fig.add_trace(go.Bar(
    x=df_stats.index, 
    y=df_stats['min'], 
    name='Mínimo'
))

fig.add_trace(go.Bar(
    x=df_stats.index, 
    y=df_stats['max'], 
    name='Máximo'
))

# Configurações do layout
fig.update_layout(
    title="Estatísticas dos Modelos (Monte Carlo - 500 rodadas)",
    xaxis_title="Modelo",
    yaxis_title="Valores",
    barmode='group',  # Para agrupar as barras
    legend_title="Estatísticas"
)

fig.show()

> Ps: Eu investiguei se o RSS tradicional e o da média de valores observáveis estavam dando os mesmos valores, e, na verdade, não estão. O que ocorre é que a variação entre eles é tão pequena que precisaria aproximar para mais de quatro casas decimais para percebê-la.

## <div align="center">Tarefa de Classificação</div>

No ambiente virtual AVA, está disposto um conjunto de dados referente aos sinais de eletromiografia, captados nos músculos faciais: Corrugador do Supercílio (Sensor 1); Zigomático Maior (Sensor 2). O presente conjunto de dados foi obtido através de um grupo de sensores chamados *Myoware Muscle Sensor*, em conjunto com um microcontrolador *NODEMCUESP32*. As aquisições foram realizadas numa taxa de amostragem de 1kHz e a resolução do ADC do microcontrolador é de 12 bits (0 – 4095). Os sensores foram posicionados em duas regiões diferentes da face de uma única pessoa, em que o primeiro sensor se encontra na região do Corrugador do Supercílio e o segundo foi posicionado no músculo Zigomático Maior. As aquisições foram realizadas seguindo um roteiro de expressões faciais focando a seguinte ordem: neutro; sorriso; sobrancelhas levantadas; surpreso; rabugento. Este roteiro se repetiu 10 vezes e cada gesto foi posto durante 1 segundo.

O arquivo *EMGDataset.csv* possui \( N = 50000 \) amostras, \( p = 2 \) características e \( C = 5 \) classes. Na primeira linha da matriz, existem os dados obtidos pelo sensor posicionado no Corrugador do Supercílio. Na segunda linha da matriz, existem os dados obtidos pelo sensor posicionado no Zigomático Maior. A terceira linha são informações referentes da categoria para cada amostra, rotuladas da seguinte maneira:

1 – Neutro  
2 – Sorriso  
3 – Sobrancelhas levantadas  
4 – Surpreso  
5 – Rabugento  

Após o download, faça o que se pede:
